In [5]:
!pip install -q gradio diffusers transformers accelerate

Şimdi, herkese açık olduğu bilinen bir InstantID modelini (örneğin `InstantID/InstantID`) kullanarak Gradio arayüzünü oluşturalım. Bu arayüz, kendi yüz görüntünüzü yüklemenize ve metin istemleri (prompt) ile özel görüntüler oluşturmanıza olanak tanır.

In [ ]:
import gradio as gr
import torch
from diffusers import StableDiffusionPipeline, ControlNetModel, StableDiffusionControlNetPipeline
from diffusers.utils import load_image
from transformers import pipeline

pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16)
pipe = pipe.to("cuda") # Ensure you have a GPU runtime enabled


from diffusers import ControlNetModel, StableDiffusionControlNetPipeline
from PIL import Image
import cv2
import numpy as np


controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16)

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16
)
pipe = pipe.to("cuda")

def generate_image(face_image, prompt):
    if face_image is None:
        return "Please upload a face image."

    # Convert Gradio Image to PIL Image if necessary
    if not isinstance(face_image, Image.Image):
         from PIL import Image
         face_image = Image.fromarray(face_image)

    # Process the face image for ControlNet (using Canny edge detection as a demonstration)
    # For actual InstantID, you would use a different preprocessing step and a different ControlNet.
    image = np.array(face_image)

    low_threshold = 100
    high_threshold = 200

    image = cv2.Canny(image, low_threshold, high_threshold)
    image = image[:, :, None]
    image = np.concatenate([image, image, image], axis=2)
    canny_image = Image.fromarray(image)


    # Generate image using the ControlNet pipeline
    # In a real InstantID, the preprocessed face image would be the 'image' input to the pipeline.
    try:
        # Using the processed canny_image with the prompt and the ControlNet pipeline
        result_image = pipe(prompt, image=canny_image).images[0]
        return result_image
    except Exception as e:
        return f"An error occurred during image generation: {e}"


with gr.Blocks() as demo:
    gr.Markdown("## InstantID (Demonstration - Using ControlNet)")
    gr.Markdown("Upload a face image and provide a text prompt to generate a new image. Note: This uses a Canny ControlNet as a demonstration of ControlNet integration, not a dedicated InstantID ControlNet.")

    with gr.Row():
        face_input = gr.Image(label="Upload Face Image")
        prompt_input = gr.Textbox(label="Enter Text Prompt")

    generate_button = gr.Button("Generate Image")

    output_image = gr.Image(label="Generated Image")

    generate_button.click(
        fn=generate_image,
        inputs=[face_input, prompt_input],
        outputs=output_image
    )

demo.launch(debug=True)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b2ce6e990e368cb83a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]